In [1]:
import numpy as np
import pandas as pd

In [2]:
from numpy import log2 as log

In [4]:
eps = np.finfo(float).eps

In [3]:
df = pd.read_csv('./data.csv',index_col = 'RID')
# df = df.drop('age', axis=1)
df

,age,income,student,credit rating,buys computer
RID,,,,,
1,young,high,no,excellent,yes
2,young,high,no,good,yes
3,middle-aged,high,no,excellent,no
4,senior,medium,no,excellent,no
5,senior,low,yes,excellent,no
6,senior,low,yes,good,yes
7,middle-aged,low,yes,good,yes
8,young,medium,no,excellent,yes
9,young,low,yes,excellent,yes


In [5]:
df['buys computer'].mode().item()

'yes'

In [6]:
def find_entropy(df):
    target = df.keys()[-1]
    entropy = 0.0
    values = df[target].unique()
    for value in np.unique(df[target]):
        fraction = df[target].value_counts()[value]/(len(df[target]))
        entropy += -fraction*np.log2(fraction)
    return entropy

In [13]:
def find_entropy_attribute(df,attribute):
    target = df.keys()[-1]
    target_variables = df[target].unique()
    variables = df[attribute].unique()
    entropy2 = 0.0
    for variable in np.unique(df[attribute]):
        entropy = 0.0
        for target_variable in np.unique(df[target]):
            num = len(df[attribute][df[attribute]==variable][df[target] == target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num / (den + eps)
            entropy+=-fraction*np.log2(fraction+eps)
        fraction2 = den/len(df)
        entropy2+=-fraction2*entropy
    return abs(entropy2)

In [14]:
def find_winner(df,att_list):
    IG = []
    for att in att_list:
        IG.append(find_entropy(df) - find_entropy_attribute(df,att))
    return att_list[np.argmax(IG)]

def get_subtable(df,node,value): # remove that column
    return df[df[node]==value].reset_index(drop=True)

In [15]:
def buildTree(df,att_list,tree=None):
    target = df.keys()[-1]
    
    if len(att_list)==0:
        
        return df[target].mode()[0]

    node = find_winner(df,att_list)
    att_list.remove(node)
    att_values = np.unique(df[node])

    if tree is None:
        tree = {}
        tree[node]={}

    for att_value in att_values:
        subtable = get_subtable(df,node,att_value) # new df
        feat_vals, counts  = np.unique(subtable[target], return_counts = True)
        if len(counts)==1: # pure
            tree[node][att_value] = feat_vals[0]
        else: 
            tree[node][att_value] = buildTree(subtable,att_list.copy())

    return tree
        
    

In [16]:
att_list = list(df.keys()[:-1])
att_list

['age', 'income', 'student', 'credit rating']

In [20]:
att_list = list(df.keys()[:-1])
t = buildTree(df, att_list)

In [18]:
import pprint
pprint.pprint(t)

{'age': {'middle-aged': {'credit rating': {'excellent': {'student': {'no': 'no',
                                                                     'yes': {'income': {'high': 'no'}}}},
                                           'good': 'yes'}},
         'senior': {'credit rating': {'excellent': 'no',
                                      'good': {'student': {'no': 'no',
                                                           'yes': 'yes'}}}},
         'young': {'income': {'high': 'yes',
                              'low': 'yes',
                              'medium': {'student': {'no': {'credit rating': {'excellent': 'no',
                                                                              'good': 'yes'}},
                                                     'yes': 'yes'}}}}}}
